In [1]:
import pandas as pd
import os

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [85]:
country_code_dict = {
    'POL':'Poland', 'NZL_NM':'New Zealand -- Non-Maori', 'FRACNP':pd.NA, 'FIN':'Finland', 'NZL_MA':'New Zealand -- Maori',
    'AUT':'Austria', 'GBR_SCO':pd.NA, 'GBR_NP':'United Kingdom', 'KOR':'Republic of Korea', 'UKR':'Ukraine',
    'CHE':'Switzerland', 'CZE':'Czechia', 'ITA':'Italy', 'AUS':'Australia', 'ISL':'Iceland',
    'SWE':'Sweden', 'SVK':'Slovakia', 'SVN':'Slovenia', 'LTU':'Lithuania', 'HRV':'Croatia',
    'BEL':'Belgium', 'GBRCENW':pd.NA, 'BLR':'Belarus', 'LUX':'Luxembourg', 'BGR':'Bulgaria',
    'DEUTNP':'Germany', 'IRL':'Ireland', 'ESP':'Spain', 'GBR_NIR':pd.NA, 'DEUTW':pd.NA,
    'GRC':'Greece', 'PRT':'Portugal', 'GBRTENW':pd.NA, 'JPN':'Japan', 'CAN':'Canada',
    'TWN':'Taiwan, Province of China', 'USA':'United States of America', 'NOR':'Norway', 'CHL':'Chile', 'HKG':'Hong Kong',
    'EST':'Estonia', 'LVA':'Latvia', 'HUN':'Hungary', 'FRATNP':'France', 'NLD':'Netherlands',
    'NZL_NP':'New Zealand', 'DNK':'Denmark', 'RUS':'Russian Federation', 'DEUTE':pd.NA, 'ISR':'Israel'
}

In [86]:
births_files = os.listdir('births/Births')
deaths_files = os.listdir('deaths/Deaths_1x1')
population_files = os.listdir('Population')

In [147]:
ret_birth = pd.DataFrame()
ret_death = pd.DataFrame()
ret_population = pd.DataFrame()
for file_name in births_files:
    birth_pd = pd.read_table(filepath_or_buffer=f'births/Births/{file_name}',
                            names=['Year', 'Female_Birth', 'Male_Birth', 'Total_Birth'],
                            sep='\s+', skiprows=3, na_values=['.'])
    birth_pd['Country'] = country_code_dict[file_name.split('.')[0]]
    ret_birth = pd.concat([ret_birth, birth_pd], ignore_index=True)
for file_name in deaths_files:
    death_pd = pd.read_table(filepath_or_buffer=f'deaths/Deaths_1x1/{file_name}',
                            names=['Year', 'Age','Female_Death', 'Male_Death', 'Total_Death'],
                            sep='\s+', skiprows=3, na_values=['.'])
    death_pd = death_pd.drop(['Age'], axis=1).groupby('Year').sum().reset_index()
    death_pd['Country'] = country_code_dict[file_name.split('.')[0]]
    ret_death = pd.concat([ret_death, death_pd], ignore_index=True)
for file_name in population_files:
    population_pd = pd.read_table(filepath_or_buffer=f'Population/{file_name}',
                            names=['Year', 'Age','Female_Population', 'Male_Population', 'Total_Population'],
                            sep='\s+', skiprows=3, na_values=['.'])
    population_pd = population_pd.drop(['Age'], axis=1).groupby('Year').sum().reset_index()
    population_pd['Country'] = country_code_dict[file_name.split('.')[0]]
    ret_population = pd.concat([ret_population, population_pd], ignore_index=True)
ret_death['Year'] = ret_death['Year'].astype('int64')
ret_population['Year'] = ret_population['Year'].astype('int64')
# ret_population = ret_population.dropna(axis=0, how='any')
ret = pd.merge(left=ret_birth, right=ret_death, on=['Country', 'Year'], how='outer')
ret = pd.merge(left=ret, right=ret_population, on=['Year', 'Country'], how='outer')

In [148]:
ret.to_csv(path_or_buf='vis.csv')